In [3]:
import pandas as pd

In [4]:
toronto=pd.read_csv('toronto_neighbourhood.csv')
toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3L,North York,Downsview West,43.739015,-79.506944
1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [54]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

## Find Toronto cooddinates and make a map of neighbourhoods

In [6]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [7]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the only boroughs with name Toronto in it. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [56]:
mylist=toronto['Borough']
sub = 'Toronto'
listing=[sub in mystring for mystring in mylist]
TorontoDF_limited=toronto[listing].reset_index(drop=True)
TorontoDF_limited.head(12)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
5,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
6,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
7,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
8,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
9,M4P,Central Toronto,Davisville North,43.712751,-79.390197


In [57]:
TorontoDF_limited.shape

(38, 5)

In [58]:
print('The new dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoDF_limited['Borough'].unique()),
        TorontoDF_limited.shape[0]
    )
)

The new dataframe has 4 boroughs and 38 neighborhoods.


## Get the Forsqure data on Toronto

In [8]:
@hidden cell CLIENT_ID = 'BUNOW5XLSZ2VMWASNKEHYYNLYPKAEEJ1G1QO05CJ34WN0JK4' # your Foursquare ID
@hidden cell CLIENT_SECRET = 'VTKQXRIOXCQV5OSTWBRZB1B5BDLWLRLCDB3OVQE2UAGMTECJ' # your Foursquare Secret
VERSION = '20192904' # Foursquare API version

### Let's explore the first neighborhood in our dataframe.
neighborhood name

In [59]:
TorontoDF_limited.loc[0, 'Neighborhood']

'Central Bay Street'

In [60]:
neighborhood_latitude = TorontoDF_limited.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = TorontoDF_limited.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = TorontoDF_limited.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


top 100 venues in Toronto

In [68]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude, 
            neighborhood_longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            neighborhood_latitude, 
            neighborhood_longitude, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [72]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=TorontoDF_limited['Latitude'],
                                   longitudes=TorontoDF_limited['Longitude']
                                  )

Downsview West
Central Bay Street
Leaside
Church and Wellesley
North Toronto West
Stn A PO Boxes 25 The Esplanade
Caledonia-Fairbanks
York Mills West
Business Reply Mail Processing Centre 969 Eastern
Thorncliffe Park
Emery, Humberlea
The Danforth West, Riverdale
Weston
Woodbine Heights
Dovercourt Village, Dufferin
Rosedale
The Kingsway, Montgomery Road, Old Mill North
Humber Summit
Cabbagetown, St. James Town
Don Mills North
Queen's Park
Parkwoods
Davisville North
Adelaide, King, Richmond
Brockton, Exhibition Place, Parkdale Village
Willowdale West
Newtonbrook, Willowdale
Forest Hill North, Forest Hill West
The Beaches West, India Bazaar
The Annex, North Midtown, Yorkville
Harbourfront, Regent Park
Northwest
Dorset Park, Scarborough Town Centre, Wexford Heights
Bayview Village
Roselawn
Downsview Central
Christie
Clarks Corners, Sullivan, Tam O'Shanter


In [70]:
len(TorontoDF_limited ['Neighborhood'].unique())

38

In [73]:
print(toronto_venues.shape) #venues near Toronto latitude and longitude coordinates (radius 500 m)
toronto_venues.head()

(3344, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downsview West,43.657952,-79.387383,Jimmy's Coffee,43.658421,-79.385613,Coffee Shop
1,Downsview West,43.657952,-79.387383,Tim Hortons,43.658570,-79.385123,Coffee Shop
2,Downsview West,43.657952,-79.387383,Hailed Coffee,43.658833,-79.383684,Coffee Shop
3,Downsview West,43.657952,-79.387383,The Queen and Beaver Public House,43.657472,-79.383524,Gastropub
4,Downsview West,43.657952,-79.387383,Panago,43.658258,-79.384313,Pizza Place


count venues in neighbourhood

In [75]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",88,88,88,88,88,88
Bayview Village,88,88,88,88,88,88
"Brockton, Exhibition Place, Parkdale Village",88,88,88,88,88,88
Business Reply Mail Processing Centre 969 Eastern,88,88,88,88,88,88
"Cabbagetown, St. James Town",88,88,88,88,88,88
Caledonia-Fairbanks,88,88,88,88,88,88
Central Bay Street,88,88,88,88,88,88
Christie,88,88,88,88,88,88
Church and Wellesley,88,88,88,88,88,88


In [76]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 54 uniques categories.


## Analyze neighbourhood

In [77]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
col_list = list(toronto_onehot.columns)
col_list.remove('Neighborhood')
fixed_columns = ['Neighborhood'] + col_list
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Museum,Bakery,Bar,Bookstore,Bubble Tea Shop,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Office,Park,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Smoothie Shop,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,Downsview West,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Downsview West,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Downsview West,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Downsview West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Downsview West,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [78]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Art Museum,Bakery,Bar,Bookstore,Bubble Tea Shop,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Office,Park,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Smoothie Shop,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
1,Bayview Village,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
2,"Brockton, Exhibition Place, Parkdale Village",0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
3,Business Reply Mail Processing Centre 969 Eastern,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
4,"Cabbagetown, St. James Town",0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
5,Caledonia-Fairbanks,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
6,Central Bay Street,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.

In [79]:
toronto_grouped.shape

(38, 55)

Show 5 top venues

In [80]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.03
4                 Bar  0.02


----Bayview Village----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.03
4                 Bar  0.02


----Brockton, Exhibition Place, Parkdale Village----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.03
4                 Bar  0.02


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3        Burger Joint  0.03
4                 Bar  0.02


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.06
2  Italian Restaurant  0.05
3   

Show each neighborhood along with the top 10 most common venues

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [82]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
1,Bayview Village,Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
4,"Cabbagetown, St. James Town",Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant


In [83]:
toronto_grouped.head()

,Neighborhood,American Restaurant,Art Museum,Bakery,Bar,Bookstore,Bubble Tea Shop,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Office,Park,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Smoothie Shop,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
1,Bayview Village,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
2,"Brockton, Exhibition Place, Parkdale Village",0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
3,Business Reply Mail Processing Centre 969 Eastern,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
4,"Cabbagetown, St. James Town",0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364


put it in a dataframe

## Cluster Neighborhoods

In [84]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [85]:
toronto_grouped_clustering.head()

,American Restaurant,Art Museum,Bakery,Bar,Bookstore,Bubble Tea Shop,Burger Joint,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Office,Park,Pizza Place,Poke Place,Portuguese Restaurant,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Smoothie Shop,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
1,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
2,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
3,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364
4,0.011364,0.011364,0.022727,0.022727,0.011364,0.022727,0.034091,0.056818,0.011364,0.022727,0.011364,0.147727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.045455,0.022727,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.022727,0.022727,0.011364,0.011364,0.022727,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364


In [86]:
toronto_grouped_clustering.shape

(38, 54)

In [88]:
toronto_merged = TorontoDF_limited
toronto_merged.rename(index=str, columns={"Neighbourhood":"Neighborhood"}, inplace=True)

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
1,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Indian Restaurant,Salad Place,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Restaurant


## Visualize the resulting clusters

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters